## Data engineering with Surf's up project

In this file we are prepping the data by doing following steps:
* Converting csv files into dataframe
* Cleaning the data - taking care of missing/NaN values
* Creating a sqlite database 
* Creating connecting with sqlite database enging and ORM classes for our data
* Creating 2 tables and storing its metadata
* Converting the data in dataframe to dictionary format so we can insert it into sqlite tables
* Inserting dictonary format data into sqlite tables and confirming the same by querying/fetching sample data from table

#### Data Handling

In [194]:
#importing all dependencies for dataframe
import pandas as pd
import numpy

In [195]:
#reading the hawaii measurements.csv files into dataframe
measure_df = pd.read_csv("hawaii_measurements.csv")
measure_df.head()
# measure_df.describe()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [196]:
#Cleaning the dataframe by replacing Nans with 0 instead of deleting rows with NaN, as the prcp values are very low 
clean_measure_df = measure_df.fillna(0)
clean_measure_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,0.00,73


In [197]:
#reading hawaii_stations.csv into dataframe
stations_df = pd.read_csv("hawaii_stations.csv")
stations_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [198]:
#saving clean measure_df to a csv file
clean_measure_df.to_csv("cleandata/clean_measurements.csv", index=False)

In [199]:
#stations_df was clean but saving it in same folder as clean measurements
stations_df.to_csv("cleandata/clean_stations.csv", index=False)

#### Creating sqlite database, engine connection and ORM classes for tables

In [200]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [201]:
# Creating an engine to a database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [202]:
#creating connection to engine
conn = engine.connect()

In [203]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
Base = declarative_base()

In [204]:
class measurements(Base):
    __tablename__ = 'measurements'
#     __table_args__ = {'extend_existing': True} # this will replacec existing metadata for measurements table/class
    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, station={self.station}, prcp={self.prcp}"

In [205]:
class stations(Base):
    __tablename__ = 'stations'
    
    id = Column(Integer, primary_key = True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f" id = {self.id}, station = {self.station}, name = {self.name}"

In [206]:
# creating 2 tables in hawaii.sqlite database using create_all
Base.metadata.create_all(engine)

In [207]:
#Checking stations table schema
stations.__table__

Table('stations', MetaData(bind=None), Column('id', Integer(), table=<stations>, primary_key=True, nullable=False), Column('station', Text(), table=<stations>), Column('name', Text(), table=<stations>), Column('latitude', Float(), table=<stations>), Column('longitude', Float(), table=<stations>), Column('elevation', Float(), table=<stations>), schema=None)

In [208]:
#checking measurements table schema
measurements.__table__

Table('measurements', MetaData(bind=None), Column('id', Integer(), table=<measurements>, primary_key=True, nullable=False), Column('station', Text(), table=<measurements>), Column('date', Text(), table=<measurements>), Column('prcp', Float(), table=<measurements>), Column('tobs', Float(), table=<measurements>), schema=None)

In [209]:
#converting clean dataframe(without NaNs) to dictionary using orient value as records (column -> value)
measure_dict = clean_measure_df.to_dict(orient='records')
print(measure_dict[0]) # checking the conversion worked ok

{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}


In [210]:
# converting stations_df to dictionary
stations_dict = stations_df.to_dict(orient='records')
print(stations_dict[0]) #checking the conversion worked ok

{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}


#### Loading dataframe dictionaries into sqlite tables

In [211]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [212]:
# creting a variable for measurements table, to load it into hawaii.sqlite later
measure_table = sqlalchemy.Table('measurements', metadata, autoload=True)

In [213]:
# creting a variable for each stations table, to load it into sqlite later
stations_table = sqlalchemy.Table('stations', metadata, autoload=True)

In [189]:
# Since I added station column later in existing measurements class, I need to delete previously existing 
# rows/data by using `table.delete()` to remove any pre-existing data and reload table
conn.execute(measure_table.delete())

In [214]:
# inserting csv data that was converted into dictionaries in tables variables created above
# Using `table.insert()` to insert the data into the table
conn.execute(measure_table.insert(), measure_dict)

In [215]:
# Test the data was inserted correctly into sqlite database by querying first 5 rows
conn.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', 0.0, 73.0)]

In [217]:
# inserting csv data that was converted into dictionaries in tables variables created above
# Using `table.insert()` to insert the data into the table
conn.execute(stations_table.insert(), stations_dict)

In [218]:
# Test the data was inserted correctly into sqlite database by querying first 5 rows
conn.execute("select * from stations limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

### Moving on to climate_analysis.ipnyb for analysis part